# Additional End of week Exercise - week 2

Now use everything you've learned from Week 2 to build a full prototype for the technical question/answerer you built in Week 1 Exercise.

This should include a Gradio UI, streaming, use of the system prompt to add expertise, and the ability to switch between models. Bonus points if you can demonstrate use of a tool!

If you feel bold, see if you can add audio input so you can talk to it, and have it respond with audio. ChatGPT or Claude can help you, or email me if you have questions.

I will publish a full solution here soon - unless someone beats me to it...

There are so many commercial applications for this, from a language tutor, to a company onboarding solution, to a companion AI to a course (like this one!) I can't wait to see your results.

# Single Function Tool

In [ ]:
# imports

import os
import json
from dotenv import load_dotenv
from openai import OpenAI
import gradio as gr

MODEL = "gpt-4o-mini"
openai = OpenAI()
openai_api_key = os.getenv('OPENAI_API_KEY')
if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")

system_message = "You are a helpful assistant for an Airline called FlightAI. "
system_message += "Give short, courteous answers, no more than 1 sentence. "
system_message += "Always be accurate. If you don't know the answer, say so."

ticket_prices = {"london": "$799", "paris": "$899", "tokyo": "$1400", "berlin": "$499"}

def get_ticket_price(destination_city):
    print(f"Tool get_ticket_price called for {destination_city}")
    city = destination_city.lower()
    return ticket_prices.get(city, "unknown")

price_function = {
    "name" : "get_ticket_price",
    "description":  "Get the price of a return ticket to the destination city. Call this whenever you need to know the ticket price, for example when a customer asks 'How much is a ticket to this city'",
    "parameters": {
        "type" : "object",
        "properties": {
            "destination_city" : {
                "type" : "string",
                "description": "The city that the customer wants to travel to"
            },
        },
        "required":["destination_city"],
        "additionalProperties":False
    }
}

tools = [{"type":"function","function": price_function}]

def handle_tool_call(message):
    tool_call = message.tool_calls[0]
    arguments = json.loads(tool_call.function.arguments)
    city = arguments.get("destination_city")
    price = get_ticket_price(city)
    response = {
        "role": "tool",
        "content": json.dumps({"destination_city": city,"price": price}),
        "tool_call_id": tool_call.id
    }
    return response, city

def chat(message, history):
    messages = [{
        "role":"system",
        "content":system_message,
        
    }] + history + [{
        "role":"user",
        "content":message
    }]
    
    response = openai.chat.completions.create(model=MODEL, messages=messages, tools=tools)
    
    if response.choices[0].finish_reason=="tool_calls":
        message = response.choices[0].message
        response, city = handle_tool_call(message)
        messages.append(message)
        messages.append(response)
        response = openai.chat.completions.create(model=MODEL, messages=messages)
        
    return response.choices[0].message.content

    

# Multiple Function Tool

In [2]:
# imports

import os
import json
from dotenv import load_dotenv
from openai import OpenAI
import gradio as gr

MODEL = "gpt-4o-mini"
openai = OpenAI()
openai_api_key = os.getenv('OPENAI_API_KEY')
if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")
    
system_message = "You are a helpful assistant for an Airline called FlightAI. "
system_message += "Give short, courteous answers, no more than 1 sentence. "
system_message += "Always be accurate. If you don't know the answer, say so."

ticket_prices = {"london": "$799", "paris": "$899", "tokyo": "$1400", "berlin": "$499"}

def get_ticket_price(destination_city):
    print(f"Tool get_ticket_price called for {destination_city}")
    city = destination_city.lower()
    return ticket_prices.get(city, "unknown")

price_function = {
    "name" : "get_ticket_price",
    "description":  "Get the price of a return ticket to the destination city. Call this whenever you need to know the ticket price, for example when a customer asks 'How much is a ticket to this city'",
    "parameters": {
        "type" : "object",
        "properties": {
            "destination_city" : {
                "type" : "string",
                "description": "The city that the customer wants to travel to"
            },
        },
        "required":["destination_city"],
        "additionalProperties":False
    }
}

def get_booking(destination_city):
    city = destination_city.lower()
    
    info = {
        "london": {"flight_number":"LD 7331", "departure_time":"10:00", "arrival_time":"12:00", "gate":"A1"},
        "paris": {"flight_number":"PA 1234", "departure_time":"14:00", "arrival_time":"16:00", "gate":"B2"},
        "tokyo": {"flight_number":"TK 4567", "departure_time":"18:00", "arrival_time":"20:00", "gate":"C3"},    
        "berlin": {"flight_number":"BE 8910", "departure_time":"22:00", "arrival_time":"00:00", "gate":"D4"}
    }
    
    if city in info:
        return f"Flight {info[city]['flight_number']} departs at {info[city]['departure_time']} and arrives at {info[city]['arrival_time']} at gate {info[city]['gate']}"
    else:    
        return "unknown"
    
booking_function = {
    "name": "get_booking",
    "description": "Get the booking details for a flight to the destination city. Call this whenever the customer need to know the booking details, for example when a customer asks 'What is my flight number?'",
    "parameters": {
        "type": "object",
        "properties": {
            "destination_city": {
                "type": "string",
                "description": "The city that the customer wants to travel to"
            },
            "departure_date": {
                "type": "string",
                "description": "The departure date in the format YYYY-MM-DD"
            },
            "return_date": {
                "type": "string",
                "description": "The return date in the format YYYY-MM-DD"
            },
            "passenger_name": {
                "type": "string",
                "description": "The passenger name"
            },
        },
        "required": ["destination_city", "departure_date", "return_date", "passenger_name"],
        "additionalProperties": False
    }
}

tools = [{"type":"function", "function": booking_function}, {"type":"function","function": price_function}]

def handle_tool_call(message):
    tool_call = message.tool_calls[0]
    arguments = json.loads(tool_call.function.arguments)
    city = arguments.get("destination_city")
    if tool_call.function.name == "get_ticket_price":
        price = get_ticket_price(city)
        response = {
            "role": "tool",
            "content": json.dumps({"destination_city": city,"price": price}),
            "tool_call_id": tool_call.id
        }
    elif tool_call.function.name == "get_booking":
        departure_date = arguments.get("departure_date")
        return_date = arguments.get("return_date")
        passenger_name = arguments.get("passenger_name")
        booking = get_booking(city)
        response = {
            "role": "tool",
            "content": json.dumps({"destination_city": city,"departure_date": departure_date, "return_date": return_date, "passenger_name": passenger_name, "booking": booking}),
            "tool_call_id": tool_call.id
        }
    return response, city

def handle_tool_call(message):
    tool_call = message.tool_calls[0]
    arguments = json.loads(tool_call.function.arguments)
    city = arguments.get("destination_city")
    price = get_ticket_price(city)
    booking = get_booking(city)
    print(booking)
    response = {
        "role": "tool",
        "content": json.dumps({"destination_city": city,"price": price, "booking": booking}),
        "tool_call_id": tool_call.id
    }
    return response, city

def chat(message, history):
    messages = [{
        "role":"system",
        "content":system_message,
        
    }] + history + [{
        "role":"user",
        "content":message
    }]
    
    response = openai.chat.completions.create(model=MODEL, messages=messages, tools=tools)
    
    if response.choices[0].finish_reason=="tool_calls":
        message = response.choices[0].message
        response, city = handle_tool_call(message)
        messages.append(message)
        messages.append(response)
        response = openai.chat.completions.create(model=MODEL, messages=messages)
        
    return response.choices[0].message.content

gr.ChatInterface(fn=chat, type="messages").launch()

OpenAI API Key exists and begins sk-proj-
* Running on local URL:  http://127.0.0.1:7869

To create a public link, set `share=True` in `launch()`.


Tool get_ticket_price called for Tokyo
Flight TK 4567 departs at 18:00 and arrives at 20:00 at gate C3


In [3]:
# imports

import os
import json
from dotenv import load_dotenv
from openai import OpenAI
import gradio as gr

MODEL = "gpt-4o-mini"
openai = OpenAI()
openai_api_key = os.getenv('OPENAI_API_KEY')
if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")
    
system_message = "You are a helpful assistant for an Airline called FlightAI. "
system_message += "Give short, courteous answers, no more than 1 sentence. "
system_message += "Always be accurate. If you don't know the answer, say so."

ticket_prices = {"london": "$799", "paris": "$899", "tokyo": "$1400", "berlin": "$499"}

def get_ticket_price(destination_city):
    print(f"Tool get_ticket_price called for {destination_city}")
    city = destination_city.lower()
    return ticket_prices.get(city, "unknown")

price_function = {
    "name" : "get_ticket_price",
    "description":  "Get the price of a return ticket to the destination city. Call this whenever you need to know the ticket price, for example when a customer asks 'How much is a ticket to this city'",
    "parameters": {
        "type" : "object",
        "properties": {
            "destination_city" : {
                "type" : "string",
                "description": "The city that the customer wants to travel to"
            },
        },
        "required":["destination_city"],
        "additionalProperties":False
    }
}

def get_booking(destination_city):
    city = destination_city.lower()
    
    info = {
        "london": {"flight_number":"LD 7331", "departure_time":"10:00", "arrival_time":"12:00", "gate":"A1"},
        "paris": {"flight_number":"PA 1234", "departure_time":"14:00", "arrival_time":"16:00", "gate":"B2"},
        "tokyo": {"flight_number":"TK 4567", "departure_time":"18:00", "arrival_time":"20:00", "gate":"C3"},    
        "berlin": {"flight_number":"BE 8910", "departure_time":"22:00", "arrival_time":"00:00", "gate":"D4"}
    }
    
    if city in info:
        return f"Flight {info[city]['flight_number']} departs at {info[city]['departure_time']} and arrives at {info[city]['arrival_time']} at gate {info[city]['gate']}"
    else:    
        return "unknown"
    
booking_function = {
    "name": "get_booking",
    "description": "Get the booking details for a flight to the destination city. Call this whenever the customer need to know the booking details, for example when a customer asks 'What is my flight number?'",
    "parameters": {
        "type": "object",
        "properties": {
            "destination_city": {
                "type": "string",
                "description": "The city that the customer wants to travel to"
            },
            "departure_date": {
                "type": "string",
                "description": "The departure date in the format YYYY-MM-DD"
            },
            "return_date": {
                "type": "string",
                "description": "The return date in the format YYYY-MM-DD"
            },
            "passenger_name": {
                "type": "string",
                "description": "The passenger name"
            },
        },
        "required": ["destination_city", "departure_date", "return_date", "passenger_name"],
        "additionalProperties": False
    }
}

tools = [{"type":"function", "function": booking_function}, {"type":"function","function": price_function}]

def handle_tool_call(message):
    tool_call = message.tool_calls[0]
    arguments = json.loads(tool_call.function.arguments)
    city = arguments.get("destination_city")
    if tool_call.function.name == "get_ticket_price":
        price = get_ticket_price(city)
        response = {
            "role": "tool",
            "content": json.dumps({"destination_city": city,"price": price}),
            "tool_call_id": tool_call.id
        }
    elif tool_call.function.name == "get_booking":
        departure_date = arguments.get("departure_date")
        return_date = arguments.get("return_date")
        passenger_name = arguments.get("passenger_name")
        booking = get_booking(city)
        response = {
            "role": "tool",
            "content": json.dumps({"destination_city": city,"departure_date": departure_date, "return_date": return_date, "passenger_name": passenger_name, "booking": booking}),
            "tool_call_id": tool_call.id
        }
    return response, city

def chat(message, history):
    messages = [{
        "role":"system",
        "content":system_message,
        
    }] + history + [{
        "role":"user",
        "content":message
    }]
    
    response = openai.chat.completions.create(model=MODEL, messages=messages, tools=tools)
    
    if response.choices[0].finish_reason=="tool_calls":
        message = response.choices[0].message
        response, city = handle_tool_call(message)
        messages.append(message)
        messages.append(response)
        response = openai.chat.completions.create(model=MODEL, messages=messages)
        
    return response.choices[0].message.content

gr.ChatInterface(fn=chat, type="messages").launch()

OpenAI API Key exists and begins sk-proj-
* Running on local URL:  http://127.0.0.1:7870

To create a public link, set `share=True` in `launch()`.


Tool get_ticket_price called for Tokyo


## Multimodal Q&A

In [19]:
translate_text("Hi", "es")


'Hola'

In [ ]:
# imports

import os
import json
from dotenv import load_dotenv
from openai import OpenAI
import anthropic
import gradio as gr
from IPython.display import Audio, display
from io import BytesIO

from gtts import gTTS
import os
import tempfile
# Initialization

load_dotenv()

openai_api_key = os.getenv('OPENAI_API_KEY')
anthropic_api_key = os.getenv("ANTHROPIC_API_KEY")

if openai_api_key:
    print("OpenAI key exists!")
else:
    print("OpenAI key not set!")

if anthropic_api_key:
    print("Anthropic API key exists!")
else:
    print("Anthropic key not set")

openai = OpenAI()

claude = anthropic.Anthropic()

openai_4o_mini_model = "gpt-4o-mini"

system_message = "You are a helpful assistant that answers tchnical questions."
system_message += "Always be accurate. If you don't know or not sure about some information, say so."


from deep_translator import GoogleTranslator

# First install deep_translator:
# pip install deep_translator

# Top 10 most spoken languages with their codes
LANGUAGES = {
    "English": "en",
    "Mandarin Chinese": "zh-CN",
    "Hindi": "hi",
    "Spanish": "es",
    "Arabic": "ar",
    "Bengali": "bn",
    "Portuguese": "pt",
    "Russian": "ru",
    "Japanese": "ja",
    "German": "de"
}

class ChatState:
    def __init__(self):
        self.speak = True
        self.target_lan = "en"
        
    def update_language(self, selected_language):
        self.target_lan = LANGUAGES.get(selected_language, None)
        
chat_state = ChatState()

def translate_text(text, target_language):
    try:
        translated_text = GoogleTranslator(source='auto', target=target_language).translate(text)
        return translated_text
    except Exception as e:
        print(f"Error translating text: {e}")
        
def talker(message):
    response = openai.audio.speech.create(
        model = "tts-1",
        voice = "onyx",
        input = message
    )

    audio_stream = BytesIO(response.content)
    output_filename = "output_audio.mp3"
    with open(output_filename, "wb") as f:
        f.write(audio_stream.read())

    #Play the generated audio
    display(Audio(output_filename, autoplay=True))
          
def chat(message, history):
    messages = [{
        "role": "system",
        "content": system_message
    }] + history + [{
        "role": "user",
        "content": message
    }]
    
    response = openai.chat.completions.create(model=openai_4o_mini_model, messages=messages)
    response_text = response.choices[0].message.content
    
    if chat_state.speak:
        talker(response_text)

    translate_message = translate_text(message, chat_state.target_lan)
    translate_response= translate_text(response_text, chat_state.target_lan)
    
    gr.Chatbot.update(value=[(translate_message, translate_response)], visible=True)
    return response.choices[0].message.content

def text_to_speech(text, lang_code):
    try:
        tts = gTTS(text=text, lang=lang_code)
        # Create temporary file
        with tempfile.NamedTemporaryFile(delete=False, suffix='.mp3') as fp:
            tts.save(fp.name)
            return fp.name
    except:
        return None
        
def process_message(message, history, speech_mode):
    messages = [{
        "role": "system",
        "content": system_message
    }] + history + [{
        "role": "user",
        "content": message
    }]
    
    response = openai.chat.completions.create(model=openai_4o_mini_model, messages=messages)
    response_text = response.choices[0].message.content
    
    audio_path = None
    if chat_state.speak:
        if speech_mode == "Translated":
            translate_response = translate_text(response_text, chat_state.target_lan)
            audio_path = text_to_speech(translate_response, chat_state.target_lan)
        else:
            talker(response_text)
        
    translate_message = translate_text(message, chat_state.target_lan)
    translate_response = translate_text(response_text, chat_state.target_lan)
    

    history_original = history + [{
        "role":"user",
        "content":message
    },
    {
        "role":"assistant",
        "content":response_text
    }]

    history_translated = [{
        "role":"user",
        "content":translate_message
    },
    {
        "role":"assistant",
        "content":translate_response
    }]
    
    return response_text, history_original, history_translated, audio_path


def update_language(selected_language):
    """
    Updates the target language for translation in the chat state.

    Args:
        selected_language (str): The language selected by the user.
        chat_state (object): The chat state object that stores the current language.
    """

    chat_state.update_language(selected_language)
    
with gr.Blocks() as demo:
    speech_mode = gr.State("Original")
    
    with gr.Row():
        speak_checkbox = gr.Checkbox(
            label="Read responses aloud",
            value=True,
            interactive=True
        )
        language_dropdown = gr.Dropdown(
            choices=list(LANGUAGES.keys()),
            value=None,
            label="Translation Language",
            interactive=True
        )
        speech_language = gr.Radio(
            choices=["Original", "Translated"],
            value="Original",
            label="Speech Language",
            interactive=True
        )
    
    # Add audio player
    audio_player = gr.Audio(label="Response Audio", visible=True)
    
    with gr.Row():
        with gr.Column():
            gr.Markdown("### Original Conversation")
            chatbot_original = gr.Chatbot(type="messages")
            msg_original = gr.Textbox(label="Message")
            send_btn = gr.Button("Send")
        
        with gr.Column():
            gr.Markdown("### Translated Conversation")
            chatbot_translated = gr.Chatbot(type="messages")
    
    state = gr.State([])
    
    def respond(message, history, current_speech_mode):

        bot_response, history_original, history_translated, audio_path = process_message(
            message, 
            history,
            current_speech_mode
        )

        return "", history_original, history_translated, audio_path
    
    send_btn.click(
        respond,
        inputs=[msg_original, state, speech_language],
        outputs=[msg_original, chatbot_original, chatbot_translated, audio_player],
    )

    def do_entry(message, history):
        history_original += [{"role":"user", "content":message}]
        history_translated += [{"role":"user", "content":message}]
        return "", history

    msg_original.submit(
        respond,
        inputs=[msg_original, state, speech_language],
        outputs=[msg_original, chatbot_original, chatbot_translated, audio_player],
    )
    
    #msg_original.submit(do_entry, inputs=[msg_original, chatbot_original,chatbot_translated], outputs=[msg_original, chatbot_original, chatbot_translated, audio_player).then(
    ##    respond, inputs=[chatbot_original,chatbot_translated ], outputs=[msg_original, chatbot_original, chatbot_translated, audio_player]
    #)
        
    #chat_state = ChatState()
    speak_checkbox.change(fn=lambda x: setattr(chat_state, 'speak', x), inputs=[speak_checkbox])
    language_dropdown.change(fn=update_language, inputs=[language_dropdown])
    
demo.launch()

OpenAI key exists!
Anthropic API key exists!
* Running on local URL:  http://127.0.0.1:7901

To create a public link, set `share=True` in `launch()`.


In [25]:
LANGUAGES['Spanish']

'es'